
# YouTube Video Data Analysis: Complete Step-by-Step Guide
## A Comprehensive Data Cleaning and Data Analysis Workflow

This notebook provides a complete walkthrough of analyzing trending video data, covering:
1. Data loading and cleaning
2. Feature engineering
3. Exploratory data analysis
4. Statistical testing
5. Country-specific insights

**Dataset**: There are two dataset provided: *USvideos.csv* and *GBvideos.csv*


## Step 1: Environment Setup (complete one step in this step)
First, we'll import all necessary libraries and configure our environment. You might need the following packages:
1. numpy
2. pandas
3. matplotlib.pyplot
4. seaborn (as sns)
5. glob
6. datetime
7. scipy

Configuration packages/settings:

9. %matplotlib inline (IPython magic command)
10. sns.set() (Seaborn configuration)
11. pd.set_option() (Pandas display options)

In [3]:
# step 1: import all modules

#------- complete the step-------------
import numpy
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import glob
import datetime
import scipy
#------- complete the step-------------

# Configuration
%matplotlib inline
sns.set(style='whitegrid', palette='muted')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.options.mode.chained_assignment = None

print("Environment setup complete!")

Environment setup complete!


## Step 2: Data Loading (complete two steps in this step)
We'll load all US videos CSV files and GB videos; and combine them into a single dataframe with country identifiers.

In [50]:
gb = pd.read_csv("GBvideos.csv")
gb.head(91)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,category,Unnamed: 12
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240.0,13548.0,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09,News,NaN
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning|interview|holly willoughby|philli...,494203,2651.0,1309.0,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09,News,NaN
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119.0,151.0,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09,News,NaN
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,1,skit|korean|language|conversation|esl|japanese...,1580028,65729.0,1529.0,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09,News,NaN
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019.0,57.0,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09,News,NaN
5,zZ2CLmvqfXg,"REVEALED - FIFA 18 stats for Chelsea's Hazard,...",Rio Ferdinand Presents FIVE,24,Rio Ferdinand|Rio|Ferdinand|5 Magazine|5 Mag|#...,317696,9449.0,135.0,464,https://i.ytimg.com/vi/zZ2CLmvqfXg/default.jpg,13.09,News,NaN
6,w8fAellnPns,Juicy Chicken Breast - You Suck at Cooking (ep...,You Suck At Cooking,26,how to|cooking|recipe|kitchen|chicken|chicken ...,479291,23935.0,638.0,1941,https://i.ytimg.com/vi/w8fAellnPns/default.jpg,13.09,News,NaN
7,LcZ2AuvxXNA,Nerf Bow Trick Shots | Dude Perfect,Dude Perfect,17,dude perfect|dude perfect stereotypes|dude per...,10532409,384841.0,7547.0,23849,https://i.ytimg.com/vi/LcZ2AuvxXNA/default.jpg,13.09,News,NaN
8,lamseWekh8Y,Weekend in BRIGHTON with Mike! | Fleur De Vlog,FleurDeVlog,22,fleurdevlog|fleur de vlog|fleurdeforce|fleur d...,65453,2761.0,33.0,223,https://i.ytimg.com/vi/lamseWekh8Y/default.jpg,13.09,News,NaN
9,QBGaO89cBMI,Radiohead - Lift,Radiohead,10,radiohead|lift|ok computer|oknotok,751743,42272.0,358.0,3250,https://i.ytimg.com/vi/QBGaO89cBMI/default.jpg,13.09,News,NaN


In [ ]:
# step 1: read all csv files into list and assign to csv_files
csv_files=[]
for files in glob.glob("*.csv"):
    csv_files.append(files)

print(f"Found {len(csv_files)} CSV files")

# Load each CSV with country code
dataframes = []

# step 2: read csv files into dataframe and append dataframe into dataframes
for file in csv_files:
    # Extract country code from filename and assign t country_code
    country_code=file[0:2]
    print(country_code)
    # read file to dataframe
    df = pd.read_csv(file)
    if df.shape[1] > 12 :
        # column concatenation
        df['category'] = df["category"].fillna('') + df.iloc[:, 12].fillna('')
        # delete unamed column after concatenation
        df = df.drop(df.columns[12], axis=1)    
    # adding a new column to the DataFrame (df) that identifies the country
    df["country"] = country_code  
    dataframes.append(df)
    
# Verify loading
print("\nSample data from first dataframe:")
display(dataframes[0].head(2))

Found 2 CSV files

Sample data from first dataframe:


,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,category,country
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240.0,13548.0,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09,News,GB
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning|interview|holly willoughby|philli...,494203,2651.0,1309.0,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09,News,GB


## Step 3: Data Cleaning (complete one function in this step)
We'll clean and standardize data types across all dataframes.

In [ ]:

"""
Step 1: define a function clean_dataframe takes a dataframe parameter. 
        the function will convert columns : ['video_id', 'title', 'channel_title', 'category_id', 'tags', 'thumbnail_link']
        into string, and handling missing Tags
"""  
def clean_dataframe(df):
    # handle string type
    for field in ['video_id', 'title', 'channel_title', 'category_id', 'tags', 'thumbnail_link']:
        if field == "tags":
            # for tags, seperate and put to list if there is any, make empty list if [none] is there
            df[field] = df[field].fillna('').astype(str).apply(lambda x: [] if x == '[none]' else x.split('|'))
        else :
            # other string field convert to string
            df[field] = df[field].fillna('').astype(str)

    for field in ['views','likes','dislikes','comment_total','date']:
        if field == "date":
            # if date, convert to string, then slice to get the first 5 char
            df[field] = df[field].astype(str).apply(lambda x: x[:5] if len(x) > 0 else 0.0)
            # convert to float64
            df[field] = df[field].astype(numpy.float64)
        else:
            # convert other remaining fields to int64
            df[field] = df[field].astype(numpy.int64)
    return df

# Clean all dataframes
cleaned_dataframes = [clean_dataframe(df) for df in dataframes]

# Verify cleaning
print("\nData types after cleaning:")
display(cleaned_dataframes[0].dtypes)



Data types after cleaning:


video_id           object
title              object
channel_title      object
category_id        object
tags               object
views               int64
likes               int64
dislikes            int64
comment_total       int64
thumbnail_link     object
date              float64
category           object
country            object
dtype: object

## Step 4: Missing Value Analysis (complete one function in this step)
We'll identify and handle any missing values in our dataset.

In [ ]:
"""
Step 1: define a function analyze_missing_data
        the function returns a missing report that contains country code, missing count and missing percent
"""

def analyze_missing_data(df_list, country_codes):
    """Analyze missing values across all dataframes"""
    pass

# defines a list contains all conuntry code.

missing_report = analyze_missing_data(cleaned_dataframes, country_codes)

# Display results
print("Missing value summary:")
print(missing_report[missing_report['missing_count'] > 0])
display(missing_report[missing_report['missing_count'] > 0])


## Step 5: Data Integration (complete five steps in this step)
Combine all country dataframes into one unified dataset.

In [ ]:
# Step 1: Combine all dataframes
#------- complete the step-------------


#------- complete the step-------------

# Step 2: Create backup before deduplication
#------- complete the step-------------


#------- complete the step-------------

# Step 3: Remove duplicate videos (keeping first occurrence) by video_id
#------- complete the step-------------


#------- complete the step-------------

# Step 4: Set video_id as index
#------- complete the step-------------


#------- complete the step-------------

# Final dataset info
print("\nCombined dataset information:")
print(f"Total videos: {len(combined_df)}")
print(f"Countries: {combined_df['country'].unique().tolist()}")


## Step 6: Feature Engineering (complete six steps in this step)
Create new features to enhance our analysis.

In [ ]:

# Engagement metrics
# Step 1: add a column named 'like_ratio', computed as likes/dislikes
#------- complete the step-------------


#------- complete the step-------------

# Step 2: add a column named 'engagement_rate', computed as (likes + dislikes + comment_total)/views
#------- complete the step-------------


#------- complete the step-------------


# Text features
# Step 3: add a column named 'title_length', computed as length of title
#------- complete the step-------------


#------- complete the step-------------

# Step 4: add a column named 'title_word_count', computed as number of words in title
#------- complete the step-------------


#------- complete the step-------------

# Step 5: add a column named 'title_has_exclamation', computed as bool value (you can make it 0 or 1) of if title contains exclamation mark
#------- complete the step-------------


#------- complete the step-------------

# Tag analysis
# Step 6: add a column named 'tags_count', computed as number of tags 
#------- complete the step-------------


#------- complete the step-------------



# Display new features
print("\nNew features created:")
display(combined_df[['like_ratio', 'engagement_rate', 
                    'title_length', 'tags_count']].describe())

## Step 7: Exploratory Data Analysis (complete two steps in this step)
Now we'll explore our dataset through visualizations and summary statistics.

In [ ]:
# Set up the figure
plt.figure(figsize=(18, 12))



# Plot 1: Views vs. Likes
# Step 1: make a dot plot shows views and likes. Take 1000 sample from all countries. x-axis will be views and y-axies will be views.
#         using different colors indicate different country. Add legend and plot title.

#------- complete the step-------------


#------- complete the step-------------

# Plot 2: Engagement Rate by Country
# Step 2: make a box plot shows different country engagement_rate

plt.tight_layout()
plt.show()

## Step 8: Country-Specific Analysis (complete one function in this step)
We'll examine trends and patterns for individual countries.

In [ ]:

# Step 1: define a function that makes bar plot of country's top 5 category's views

def analyze_country(df, country_code):
    """Generate country-specific analysis"""

    #------- complete the step-------------


    #------- complete the step-------------
    pass

# Analyze sample countries
for country in ['US', 'GB']:
    analyze_country(combined_df, country)

## Step 9: Advanced Insights (complete one step in this step)
We'll extract deeper insights through more sophisticated analysis.

In [ ]:
# Step 1: can you think of a analysis and visualize it?


## Step 10: Saving Results
Finally, we'll save our cleaned data and visualizations.

In [ ]:
# Step 1: Save cleaned data into csv file
#------- complete the step-------------


#------- complete the step-------------

print("Saved cleaned data to 'cleaned_youtube_trending_data.csv'")

# Step 2: Save top 5 globa category bar plot visualizations
#------- complete the step-------------


#------- complete the step-------------

print("Saved visualization to 'top_categories.png'")
